# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    text_freq = Counter(text)
    text_freq_sort = list(word for word,freq in text_freq.most_common())
    int_to_vocab = dict(enumerate(text_freq_sort))
    vocab_to_int = {word:number for number, word in int_to_vocab.items()}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punctuation_dictionary_biaodianfuhao = {
        '.':'||PERIOD||',
        ',':'||COMMA||',
        '"':'||QUOTATION_MARK||',
        ';':'||SEMICOLON||',
        '!':'||EXCLAMATION_MASK||',
        '?':'||QUESTION_MARK||',
        '(':'||LEFT_PARENTHESES||',
        ')':'||RIGHT_PARENTHESES||',
        '-':'||DASH||',
        '\n':'||RETURN||'
    }    
    return punctuation_dictionary_biaodianfuhao

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [7]:
print('Unique words: ', len(vocab_to_int))
print('Length: ',len(int_text))

Unique words:  21388
Length:  892110


In [8]:
print('Tokenized review: \n', int_text[:100])

Tokenized review: 
 [24, 22, 47, 1, 1, 1, 17, 47, 22, 82, 20, 6, 1252, 545, 8782, 7189, 20, 241, 1, 149, 1, 1, 1, 84, 4, 200, 238, 149, 208, 58, 55, 135, 64, 47, 3, 24, 22, 18, 677, 208, 58, 1, 1, 1, 24, 220, 126, 2, 121, 50, 47, 86, 2, 26, 82, 22, 289, 1, 45, 82, 374, 62, 22, 289, 2, 121, 50, 47, 10, 76, 48, 149, 271, 8, 248, 191, 2, 65, 204, 27, 128, 55, 48, 1, 11, 28, 98, 2744, 172, 12, 84, 4, 1366, 3, 2, 5, 388, 248, 69, 1]


## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [11]:
from torch.utils.data import TensorDataset, DataLoader

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    seq_len = sequence_length
    feature_x = np.array([words[index1: index1 + seq_len] for index1, number1 in enumerate(words[0: -seq_len])])
    target_y = np.array([words[index2 + seq_len] for index2, number2 in enumerate(words[0: -seq_len])])
    feature_tensor = torch.from_numpy(feature_x)
    target_tensor = torch.from_numpy(target_y)
    dataset_tensor = TensorDataset(feature_tensor, target_tensor)
    data_loader = DataLoader(dataset_tensor, shuffle=True, batch_size=batch_size)
    
    # return a dataloader
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [12]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 27,  28,  29,  30,  31],
        [ 36,  37,  38,  39,  40],
        [ 42,  43,  44,  45,  46],
        [ 37,  38,  39,  40,  41],
        [ 18,  19,  20,  21,  22],
        [ 22,  23,  24,  25,  26],
        [ 16,  17,  18,  19,  20],
        [ 32,  33,  34,  35,  36],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15]])

torch.Size([10])
tensor([ 32,  41,  47,  42,  23,  27,  21,  37,  39,  16])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [14]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.output_size = output_size
        self.hidden_dimension = hidden_dim
        self.layers_num = n_layers        
        
        # define model layers
        # 1. convert text to word2vec embedding vectors
        self.embedding_convert = nn.Embedding(vocab_size, embedding_dim)
        
        # 2. Use LSTM as the memory part choice instead of GRU and RNN because of I have used it in former class
        self.lstm_model = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        
        # 3. Use dropout to reduce the probability of overfitting
        self.drop = nn.Dropout(0.3)
        self.full_connect = nn.Linear(hidden_dim, output_size)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        batch_size = nn_input.size(0)
        embedded = self.embedding_convert(nn_input)
        lstm_result, hidden = self.lstm_model(embedded, hidden)
        lstm_result_contiguous = lstm_result.contiguous().view(-1, self.hidden_dimension)
        lstm_drop_output = self.drop(lstm_result_contiguous)
        lstm_full_connect_output = self.full_connect(lstm_drop_output)
        lstm_reshape_output = lstm_full_connect_output.view(batch_size, -1, self.output_size)
        output = lstm_reshape_output[:,-1]
        
        # return one batch of output word scores and the hidden state
        return output, hidden
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.layers_num, batch_size, self.hidden_dimension).zero_().cuda(),
                  weight.new(self.layers_num, batch_size, self.hidden_dimension).zero_().cuda())
        else:
            hidden = (weight.new(self.layers_num, batch_size, self.hidden_dimension).zero_(),
                      weight.new(self.layers_num, batch_size, self.hidden_dimension).zero_())
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [15]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    clip = 5 
    
    # move data to GPU, if available
    if(train_on_gpu):
        inp, target = inp.cuda(),target.cuda()
    # perform backpropagation and optimization
    h = tuple([each.data for each in hidden])
    rnn.zero_grad()
    output, h = rnn(inp, h)
    loss = criterion(output, target)
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    optimizer.step()
    
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [15]:
# Data params
# Sequence Length
sequence_length = 5  # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [16]:
# Training parameters
# Number of Epochs
num_epochs = 5
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 200
# Hidden Dimension
hidden_dim = 128
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 5 epoch(s)...
Epoch:    1/5     Loss: 5.736356316566467

Epoch:    1/5     Loss: 5.0695171866416935

Epoch:    1/5     Loss: 4.8759437255859375

Epoch:    1/5     Loss: 4.765796272754669

Epoch:    1/5     Loss: 4.6524876370429995

Epoch:    1/5     Loss: 4.609727342128753

Epoch:    1/5     Loss: 4.552313428878784

Epoch:    1/5     Loss: 4.508280675888061

Epoch:    1/5     Loss: 4.469550525188446

Epoch:    1/5     Loss: 4.451045530319214

Epoch:    1/5     Loss: 4.435112231731415

Epoch:    1/5     Loss: 4.407603314876557

Epoch:    1/5     Loss: 4.395122224330902

Epoch:    2/5     Loss: 4.320957453012221

Epoch:    2/5     Loss: 4.253369105815888

Epoch:    2/5     Loss: 4.265791108608246

Epoch:    2/5     Loss: 4.264162052631378

Epoch:    2/5     Loss: 4.243436612129211

Epoch:    2/5     Loss: 4.2293019537925725

Epoch:    2/5     Loss: 4.241061078071594

Epoch:    2/5     Loss: 4.23615146780014

Epoch:    2/5     Loss: 4.239074834346772

Epoch:    2/5     Loss: 

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


In [18]:
sequence_length = 8
batch_size = 128
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 5
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 200
hidden_dim = 128
n_layers = 2
show_every_n_batches = 500
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 5 epoch(s)...
Epoch:    1/5     Loss: 5.700520219802857

Epoch:    1/5     Loss: 5.074619234085083

Epoch:    1/5     Loss: 4.87677560043335

Epoch:    1/5     Loss: 4.763863982200623

Epoch:    1/5     Loss: 4.64685453748703

Epoch:    1/5     Loss: 4.590786011219024

Epoch:    1/5     Loss: 4.548758548736572

Epoch:    1/5     Loss: 4.502324135780334

Epoch:    1/5     Loss: 4.465756823539734

Epoch:    1/5     Loss: 4.457127519130707

Epoch:    1/5     Loss: 4.420526689529419

Epoch:    1/5     Loss: 4.408765558719635

Epoch:    1/5     Loss: 4.3743107862472534

Epoch:    2/5     Loss: 4.314817929538534

Epoch:    2/5     Loss: 4.271674827098846

Epoch:    2/5     Loss: 4.257754339694976

Epoch:    2/5     Loss: 4.226168549060821

Epoch:    2/5     Loss: 4.216335472106934

Epoch:    2/5     Loss: 4.2272653136253355

Epoch:    2/5     Loss: 4.199478821277618

Epoch:    2/5     Loss: 4.203442573547363

Epoch:    2/5     Loss: 4.211216173648834

Epoch:    2/5     Loss: 4.2

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [19]:
sequence_length = 10
batch_size = 128
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 5
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 200
hidden_dim = 128
n_layers = 2
show_every_n_batches = 500
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 5 epoch(s)...
Epoch:    1/5     Loss: 5.741761904716491

Epoch:    1/5     Loss: 5.097426958084107

Epoch:    1/5     Loss: 4.891838965415954

Epoch:    1/5     Loss: 4.796675529956818

Epoch:    1/5     Loss: 4.679732998847961

Epoch:    1/5     Loss: 4.65690816116333

Epoch:    1/5     Loss: 4.565067084789276

Epoch:    1/5     Loss: 4.537825615406036

Epoch:    1/5     Loss: 4.500740071773529

Epoch:    1/5     Loss: 4.456955916881562

Epoch:    1/5     Loss: 4.420249331951141

Epoch:    1/5     Loss: 4.3992762355804444

Epoch:    1/5     Loss: 4.379688757419586

Epoch:    2/5     Loss: 4.3259988118989545

Epoch:    2/5     Loss: 4.28818502664566

Epoch:    2/5     Loss: 4.260167226791382

Epoch:    2/5     Loss: 4.252535111427307

Epoch:    2/5     Loss: 4.2513523168563845

Epoch:    2/5     Loss: 4.247805739879608

Epoch:    2/5     Loss: 4.232141211509704

Epoch:    2/5     Loss: 4.213584925174713

Epoch:    2/5     Loss: 4.231984965801239

Epoch:    2/5     Loss: 4.

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [20]:
sequence_length = 12
batch_size = 128
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 5
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 200
hidden_dim = 128
n_layers = 2
show_every_n_batches = 500
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 5 epoch(s)...
Epoch:    1/5     Loss: 5.76348767375946

Epoch:    1/5     Loss: 5.092492949485779

Epoch:    1/5     Loss: 4.913378890991211

Epoch:    1/5     Loss: 4.744619331359863

Epoch:    1/5     Loss: 4.704954726219177

Epoch:    1/5     Loss: 4.604330915927887

Epoch:    1/5     Loss: 4.565940848350525

Epoch:    1/5     Loss: 4.505078238487243

Epoch:    1/5     Loss: 4.477114411830902

Epoch:    1/5     Loss: 4.461080344200134

Epoch:    1/5     Loss: 4.4206434984207155

Epoch:    1/5     Loss: 4.399798144340515

Epoch:    1/5     Loss: 4.403091289997101

Epoch:    2/5     Loss: 4.33245626888531

Epoch:    2/5     Loss: 4.274852204799652

Epoch:    2/5     Loss: 4.261784801006317

Epoch:    2/5     Loss: 4.2365141868591305

Epoch:    2/5     Loss: 4.234576435565948

Epoch:    2/5     Loss: 4.259937201023102

Epoch:    2/5     Loss: 4.234628213882447

Epoch:    2/5     Loss: 4.220967360973358

Epoch:    2/5     Loss: 4.216181794166565

Epoch:    2/5     Loss: 4.1

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## Sequence_length comparison:
I tried seq_len to be 5, 8, 10, 12 with other parameters constant. Through comparing with the average loss of last 5 and 10 steps, I found seq_len=8 performs the best. The ave_loss of last 5 steps is 3.983126. The ave_loss of last 10 steps is 3.982271.   

In [21]:
sequence_length = 8 # fixed
batch_size = 64
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 5
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 200
hidden_dim = 256
n_layers = 2
show_every_n_batches = 500
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 5 epoch(s)...
Epoch:    1/5     Loss: 5.612438001155853

Epoch:    1/5     Loss: 5.048959288597107

Epoch:    1/5     Loss: 4.924030892372131

Epoch:    1/5     Loss: 4.785824496746063

Epoch:    1/5     Loss: 4.7098476476669315

Epoch:    1/5     Loss: 4.633303977489471

Epoch:    1/5     Loss: 4.595341374397278

Epoch:    1/5     Loss: 4.541010577201844

Epoch:    1/5     Loss: 4.508524628162384

Epoch:    1/5     Loss: 4.4835720791816716

Epoch:    1/5     Loss: 4.441730164051056

Epoch:    1/5     Loss: 4.444182185649872

Epoch:    1/5     Loss: 4.438877734661102

Epoch:    1/5     Loss: 4.393205742835999

Epoch:    1/5     Loss: 4.380232720375061

Epoch:    1/5     Loss: 4.397461714267731

Epoch:    1/5     Loss: 4.378520568370819

Epoch:    1/5     Loss: 4.3642884492874146

Epoch:    1/5     Loss: 4.3289724588394165

Epoch:    1/5     Loss: 4.291635856151581

Epoch:    1/5     Loss: 4.299943231105805

Epoch:    1/5     Loss: 4.310223547458649

Epoch:    1/5     Loss:

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [23]:
sequence_length = 8 # fixed
batch_size = 256
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 5
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 200
hidden_dim = 256
n_layers = 2
show_every_n_batches = 500
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 5 epoch(s)...
Epoch:    1/5     Loss: 5.450597478866577

Epoch:    1/5     Loss: 4.7913807134628295

Epoch:    1/5     Loss: 4.589570858478546

Epoch:    1/5     Loss: 4.465808655261993

Epoch:    1/5     Loss: 4.385310161113739

Epoch:    1/5     Loss: 4.3342368516922

Epoch:    2/5     Loss: 4.229103730703757

Epoch:    2/5     Loss: 4.15231340789795

Epoch:    2/5     Loss: 4.133689846038818

Epoch:    2/5     Loss: 4.110634082317352

Epoch:    2/5     Loss: 4.104037349700928

Epoch:    2/5     Loss: 4.075345734596253

Epoch:    3/5     Loss: 4.013624293290503

Epoch:    3/5     Loss: 3.959526354789734

Epoch:    3/5     Loss: 3.9496195254325865

Epoch:    3/5     Loss: 3.9563500876426696

Epoch:    3/5     Loss: 3.9544570355415343

Epoch:    3/5     Loss: 3.9315961956977845

Epoch:    4/5     Loss: 3.892719566822052

Epoch:    4/5     Loss: 3.8297330598831176

Epoch:    4/5     Loss: 3.841893100261688

Epoch:    4/5     Loss: 3.8563372468948365

Epoch:    4/5     Loss:

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## Batch_size comparison:
After fixing seq_len as 8, I tried batch_size to be 64, 128, 256 with other parameters constant. Through comparing with the  loss performance in 5 epoches, I found batch_size=256 performs the best. 
* The final loss of batch_size=64 is 4.00681697368621
* The final loss of batch_size=128 is 3.96596155405044
* The final loss of batch_size=256 is 3.77990677642822

In [24]:
sequence_length = 8 # fixed
batch_size = 256 # fixed
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 5
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 200
hidden_dim = 128
n_layers = 2
show_every_n_batches = 500
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 5 epoch(s)...
Epoch:    1/5     Loss: 5.627948972702026

Epoch:    1/5     Loss: 4.986702287673951

Epoch:    1/5     Loss: 4.77150483417511

Epoch:    1/5     Loss: 4.635304685592652

Epoch:    1/5     Loss: 4.560186540603637

Epoch:    1/5     Loss: 4.46991529417038

Epoch:    2/5     Loss: 4.386831776882575

Epoch:    2/5     Loss: 4.306079657554626

Epoch:    2/5     Loss: 4.287217292308807

Epoch:    2/5     Loss: 4.268886025428772

Epoch:    2/5     Loss: 4.2292362160682675

Epoch:    2/5     Loss: 4.2199700512886045

Epoch:    3/5     Loss: 4.174347044491187

Epoch:    3/5     Loss: 4.128111271858216

Epoch:    3/5     Loss: 4.09560781955719

Epoch:    3/5     Loss: 4.091390362739563

Epoch:    3/5     Loss: 4.104183988571167

Epoch:    3/5     Loss: 4.1122426905632015

Epoch:    4/5     Loss: 4.0503169065568505

Epoch:    4/5     Loss: 4.002604823112488

Epoch:    4/5     Loss: 4.025039931297302

Epoch:    4/5     Loss: 4.019179473400116

Epoch:    4/5     Loss: 4.

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [25]:
sequence_length = 8 # fixed
batch_size = 256 # fixed
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 5
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 200
hidden_dim = 512
n_layers = 2
show_every_n_batches = 500
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 5 epoch(s)...
Epoch:    1/5     Loss: 5.21010164642334

Epoch:    1/5     Loss: 4.592617632865906

Epoch:    1/5     Loss: 4.424833781242371

Epoch:    1/5     Loss: 4.322242027282715

Epoch:    1/5     Loss: 4.240871015548706

Epoch:    1/5     Loss: 4.189500706672669

Epoch:    2/5     Loss: 4.0875603774698765

Epoch:    2/5     Loss: 3.9978622374534605

Epoch:    2/5     Loss: 3.9868872723579405

Epoch:    2/5     Loss: 3.9843822827339173

Epoch:    2/5     Loss: 3.9552304577827453

Epoch:    2/5     Loss: 3.940020643234253

Epoch:    3/5     Loss: 3.8734167969323754

Epoch:    3/5     Loss: 3.795744110584259

Epoch:    3/5     Loss: 3.783499454498291

Epoch:    3/5     Loss: 3.803748380661011

Epoch:    3/5     Loss: 3.800052216529846

Epoch:    3/5     Loss: 3.8023596167564393

Epoch:    4/5     Loss: 3.7194953863698292

Epoch:    4/5     Loss: 3.6500295729637147

Epoch:    4/5     Loss: 3.6707928586006164

Epoch:    4/5     Loss: 3.685105306625366

Epoch:    4/5     

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## Hidden_dim comparison:
After fixing seq_len as 8 and batch_size as 256, I tried hidden_dim to be 128, 256, 512 with other parameters constant. Through comparing with the loss performance in 5 epoches, I found hidden_dim=512 performs the best in loss. However, the speed of hidden_dim=512 is slow. Therefore, the compromised hidden_dim=256 may be a better choice to try on bigger epoches. 
* The final loss of hidden_dim=128 is 3.94509029245376
* The final loss of hidden_dim=256 is 3.77990677642822
* The final loss of hidden_dim=512 is 3.6035457406044

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** 
* I tried seq_len to be 5, 8, 10, 12 with other parameters constant. Through comparing with the average loss of last 5 and 10 steps, I found seq_len=8 performs the best. The ave_loss of last 5 steps is 3.983126. The ave_loss of last 10 steps is 3.982271
* Fixing seq_len as 8, I tried batch_size to be 64, 128, 256 with other parameters constant. Through comparing with the  loss performance in 5 epoches, I found batch_size=256 performs the best. The final loss of batch_size=256 is 3.77990677642822
* Fixing seq_len as 8 and batch_size as 256, I tried hidden_dim to be 128, 256, 512 with other parameters constant. Through comparing with the loss performance in 5 epoches, I found hidden_dim=512 performs the best in loss. However, the speed of hidden_dim=512 is slow. Therefore, the compromised hidden_dim=256 may be a better choice to try on bigger epoches. The final loss of hidden_dim=256 is 3.77990677642822
* From hyperparameters class, I learnt that layers to be at least 2 is beneficial to model performance, but 3 layers don't perform significantly better. Therefore, I choose n_layers=2.
* My Hyperparameters: seq_len=8, batch_size=256, hidden_dim=256, n_layers=2.

In [30]:
from workspace_utils import active_session

with active_session():
    sequence_length = 8 # fixed
    batch_size = 256 # fixed
    train_loader = batch_data(int_text, sequence_length, batch_size)
    num_epochs = 20
    learning_rate = 0.001
    vocab_size = len(vocab_to_int)
    output_size = vocab_size
    embedding_dim = 200
    hidden_dim = 256 # fixed
    n_layers = 2
    show_every_n_batches = 500
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    if train_on_gpu:
        rnn.cuda()
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)
    helper.save_model('./save/trained_rnn', trained_rnn)
    print('Model Trained and Saved')

Training for 20 epoch(s)...
Epoch:    1/20    Loss: 5.515294498443604

Epoch:    1/20    Loss: 4.820821399211884

Epoch:    1/20    Loss: 4.634312658309937

Epoch:    1/20    Loss: 4.5014155812263486

Epoch:    1/20    Loss: 4.429888867855072

Epoch:    1/20    Loss: 4.371651630878448

Epoch:    2/20    Loss: 4.268926535437747

Epoch:    2/20    Loss: 4.1865960011482235

Epoch:    2/20    Loss: 4.166085233688355

Epoch:    2/20    Loss: 4.1168764276504515

Epoch:    2/20    Loss: 4.1079344615936275

Epoch:    2/20    Loss: 4.127254176616669

Epoch:    3/20    Loss: 4.035472831105798

Epoch:    3/20    Loss: 3.989614639759064

Epoch:    3/20    Loss: 3.989317009449005

Epoch:    3/20    Loss: 3.970221293926239

Epoch:    3/20    Loss: 3.9570244975090025

Epoch:    3/20    Loss: 3.9770770802497863

Epoch:    4/20    Loss: 3.907030669654288

Epoch:    4/20    Loss: 3.882734680175781

Epoch:    4/20    Loss: 3.8691995701789854

Epoch:    4/20    Loss: 3.860014687538147

Epoch:    4/20    L

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [31]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [34]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: klein: girls: bass: you know what you think?

jerry: i was a very good man.

george: i think you're gonna be in the bathroom.(george enters)

elaine: oh. i got to get my coat.

jerry: i know.

jerry:(pointing) you know what? i mean, i'm going to be honest with her...

jerry: oh, yeah, well you don't have any money to get a big salad on my head.

elaine: well what about you?

jerry: no!

elaine: i was just curious, i was thinking. i was just trying to get a little too seriously.

jerry:(leaving) yeah?

jerry: what about you?

elaine: i don't know.(to elaine) i don't know, i was just curious. i mean, i know, i can't get it.

jerry: i know.

jerry:(to jerry) hey, i can't believe this is the best way to do it.

newman:(to george) i don't know what it is.

kramer: oh, hi, jerry.

kramer: yeah.

elaine:(looking at the phone) yeah, you see, i gotta be honest with you.

jerry: you know what? you know, if i could have the best of the same time.(to the door) oh, i got a lot of people to g

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [35]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.